In [32]:
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout, InputLayer

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/microsoft-stock-time-series-analysis/Microsoft_Stock.csv


In [33]:
df = pd.read_csv("/kaggle/input/microsoft-stock-time-series-analysis/Microsoft_Stock.csv")
for col in df.columns:
    df = df[pd.isnull(df[col])==False]
    
df['Date'] = df['Date'].apply(lambda x: x.split()[0])
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df['Average Price'] = df[['Open', 'High', 'Low', 'Close']].mean(axis=1)

price = df['Average Price']

In [34]:
xaxis_train = df.index[df.index <= '2020']
xaxis_test = df.index[df.index >= '2020']
yaxis_train = df['Average Price'][:'2020']
yaxis_test = df['Average Price']['2020':]

fig = go.Figure()

fig.add_trace(go.Scatter(x=xaxis_train,
                         y=yaxis_train,
                         name='Training set (Before 2020)'))
fig.add_trace(go.Scatter(x=xaxis_test,
                         y=yaxis_test,
                         name='Test set (2020 and beyond)'))

fig.update_layout(
    title='Microsoft stock price',
    xaxis_title='Date',
    yaxis_title='Price',
    legend_title='Data sets'
)

fig.show()

In [35]:
class DataProcessor:
    def __init__(self, timesteps):
        self.timesteps = timesteps

    def process_data(self, data):
        df_as_np = data.to_numpy()
        X = []
        y = []

        for i in range(len(df_as_np) - self.timesteps):
            row = [[a] for a in df_as_np[i:i + self.timesteps]]
            X.append(row)
            label = df_as_np[i + self.timesteps]
            y.append(label)

        X = np.array(X)
        y = np.array(y)

        return X, y

class Scale:
    def __init__(self):
        self.scaler = MinMaxScaler(feature_range=(0, 1))

    def scale_data(self, data):
        scaled_data = self.scaler.fit_transform(data.reshape(-1, 1))
        return self.scaler, scaled_data

In [36]:
processor = DataProcessor(90)
X, y = processor.process_data(price)
scaler, scaled_data = Scale().scale_data(y)
X_train, y_train = X[:1100], scaled_data[:1100]
X_val, y_val = X[1100:1200], scaled_data[1100:1200]
X_test, y_test = X[1200:], scaled_data[1200:]

In [38]:
class GRUModel(Sequential):
    def __init__(self):
        super().__init__()
        self.add(InputLayer((90, 1)))
        self.add(GRU(64))
        self.add(Dense(1, activation='tanh'))

model = GRUModel()
model.summary()

Model: "gru_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_2 (GRU)                 (None, 64)                12864     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 12,929
Trainable params: 12,929
Non-trainable params: 0
_________________________________________________________________


In [39]:
cp = ModelCheckpoint('model/', save_best_only=True)
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[cp])

Epoch 1/100
35/35 [==============================] - 6s 116ms/step - loss: 1.1940 - root_mean_squared_error: 1.0927 - val_loss: 1.7349 - val_root_mean_squared_error: 1.3172
Epoch 2/100
35/35 [==============================] - 3s 102ms/step - loss: 0.7826 - root_mean_squared_error: 0.8846 - val_loss: 0.6999 - val_root_mean_squared_error: 0.8366
Epoch 3/100
35/35 [==============================] - 4s 103ms/step - loss: 0.1535 - root_mean_squared_error: 0.3918 - val_loss: 0.0450 - val_root_mean_squared_error: 0.2120
Epoch 4/100
35/35 [==============================] - 4s 103ms/step - loss: 0.0063 - root_mean_squared_error: 0.0795 - val_loss: 0.0096 - val_root_mean_squared_error: 0.0981
Epoch 5/100
35/35 [==============================] - 4s 130ms/step - loss: 0.0015 - root_mean_squared_error: 0.0381 - val_loss: 0.0071 - val_root_mean_squared_error: 0.0842
Epoch 6/100
35/35 [==============================] - 3s 102ms/step - loss: 8.9059e-04 - root_mean_squared_error: 0.0298 - val_loss: 0.0

In [40]:
mse, rmse = model.evaluate(X_test, y_test)
print("Mean Squared Error on test set: ", mse)
print("Root Mean Squared Error on test set: ", rmse)

7/7 [==============================] - 0s 3ms/step - loss: 0.0405 - root_mean_squared_error: 0.2013
Mean Squared Error on test set:  0.040511928498744965
Root Mean Squared Error on test set:  0.2012757509946823


In [52]:
predicted_train = model.predict(X_train)
predicted_train = scaler.inverse_transform(predicted_train)
predicted_test = model.predict(X_test)
predicted_test = scaler.inverse_transform(predicted_test)

fig = go.Figure()

fig.add_trace(go.Scatter(x=xaxis_train,
                         y=yaxis_train,
                         mode='lines',
                         name='Training set (Before 2020)',
                         line_color='blue'))

fig.add_trace(go.Scatter(x=xaxis_test,
                         y=yaxis_test,
                         mode='lines',
                         name='Test set (2020 and beyond)',
                         line_color='blue'))

fig.add_trace(go.Scatter(x=xaxis_test[90:],
                         y=predicted_test.flatten(),
                         mode='lines',
                         name='Predicted Prices (Test)',
                         line_color='red'))

fig.update_layout(
    title='Microsoft stock price and predictions',
    xaxis_title='Date',
    yaxis_title='Price',
    legend_title='Data sets'
)

fig.show()

7/7 [==============================] - 0s 3ms/step
